강화학습을 구성하는 여러 요소 중 3가지는 아래와 같다.

1. 액션 의존성- Agent가 하는 action에 따라 받는 reward가 다르다.

2. 시간 의존성- action에 대한 reward를 바로바로 받는 것이 아니다. 얼마의 시간이 경과한 뒤 reward를 받을 수도 있고 sparse하다.

3. 상태 의존성- Environment의 상태에 따라 받는 reward가 다르다.

이번 예시는 bandit에 상태정보가 추가된 **컨텍스트 밴딧**문제로서, 2번만을 고려하지 않는다.

아래는 강화학습 과정을 나타낸다.

1. Agent는 Environment로부터 받은 상태를 기반으로 action을 생산한다. (단, 이번 Bandit 예제에서는 Environement의 상태는 없다.)

2. Environment는 Agent로부터 받은 action을 기반으로 reward를 생산한다. 그리고 자신을 그 action을 기반으로 상태를 업데이트 한다. (단, Environment의 상태가 없기 때문에 생략한다.)

3. Agent는 앞서 생산했던 action을 주고 어떤 reward를 받았는지를 통해 학습을 진행한다.

In [1]:
import numpy as np

class Environment:
    def __init__(self, bandit_arms):
        self.bandit_arms = list(bandit_arms)
        self.num_arms = len(bandit_arms)

    def pull_bandit(self, action):
        result = np.random.randn()
        return 1 if result > self.bandit_arms[action] else -1

bandits = [
    Environment([0.2, 0, -0.2, -2]),
    Environment([0.1, -5, 0, 2]),
    Environment([-2, 1, -1, 0.25]),
]

In [2]:
import torch

class Agent():
    def __init__(self, num_actions, num_bandits):
        self.num_bandits = num_bandits
        self.policy = torch.nn.Sequential(
            torch.nn.Linear(num_bandits, num_actions, False),
            torch.nn.Softmax(dim=0),
        )
        torch.nn.init.ones_(self.policy[0].weight)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=1e-3)

    def action(self, state:int) -> int:
        with torch.inference_mode():
            state = torch.tensor(state)
            state = torch.nn.functional.one_hot(state, num_classes=self.num_bandits).float()
            output = self.policy(state)
            action = torch.multinomial(output, 1)
        return action.detach().item()

    def train(self, action, state, reward):
        responsible_output = self.policy[0].weight[action, state]
        loss = -reward * torch.log(responsible_output)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

agent = Agent(4, 3)

In [3]:
total_episode = 10000
total_reward = np.zeros((4, 3))

for i in range(total_episode):
    bandit_index = np.random.randint(3)
    bandit = bandits[bandit_index]
    action = agent.action(bandit_index)
    reward = bandit.pull_bandit(action)
    agent.train(action, bandit_index, reward)
    total_reward[action, bandit_index] += reward
    if i % 500 == 499:
        print(i + 1, total_reward.mean(axis=0))

500 [10.   4.   7.5]
1000 [24.   5.5 20. ]
1500 [34.25 11.75 34.5 ]
2000 [47.5 19.5 56. ]
2500 [64.25 33.25 76.  ]
3000 [83.  50.5 95. ]
3500 [107.75  69.   115.75]
4000 [128.25  96.5  143.75]
4500 [148.   120.25 168.75]
5000 [170.75 148.   202.75]
5500 [195.75 181.25 236.  ]
6000 [219.25 215.25 265.5 ]
6500 [251.75 245.5  297.25]
7000 [281.5 281.  330. ]
7500 [317.25 313.   365.25]
8000 [352.5 351.  397. ]
8500 [384.   393.75 428.25]
9000 [418.   429.25 459.75]
9500 [448.  473.5 501. ]
10000 [488.75 507.5  540.75]


In [4]:
agent.policy[0].weight

Parameter containing:
tensor([[0.7053, 0.9026, 4.6664],
        [1.0883, 5.2763, 0.0273],
        [1.3529, 0.9433, 2.8910],
        [4.8534, 0.0183, 0.7820]], requires_grad=True)